In [113]:

import gym
from gym import logger as gymlogger
from gym.wrappers import RecordVideo
gymlogger.set_level(40) #error only
#import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")

In [114]:
env = gym.make("CartPole-v1")

## Initialize variables

In [115]:
#LEARNING_RATE = 0.1
LEARNING_RATE = 0.1

DISCOUNT = 0.99 #instead of 0.95
EPISODES = 60000
total = 0
total_reward = 0

#0.25, 0.25, 0.01, 0.1
#Observation = [30, 30, 50, 50]
Observation = [30, 30, 50, 50]
#np_array_win_size = np.array([0.1, 0.1, 0.01, 0.1])
np_array_win_size = np.array([0.1, 0.1, 0.01, 0.1])

epsilon = 0.9
epsilon_decay = 0.995
epsilon_min = 0.1
epsilon_max = 0.95

prev_mean = 0
obs = env.reset()


new_env = env.step(0)[0]
print(new_env)
print(round(new_env[2], 2))

[-0.01986111 -0.2092384  -0.05000179  0.23621972]
-0.05


## Create Q table

In [116]:
q_table = np.zeros((Observation + [env.action_space.n]))
q_table.shape

(30, 30, 50, 50, 2)

## Getting discrete state

In [117]:
def get_discrete_state(state):
    
    #Given a state, round to the nearest 0.25
    #divide by the window size to find which bucket it belongs to
    #discrete_state =np.array([np.round(state[0]*4)/4, np.round(state[1] *4) /4, np.round(state[2], 1) ,np.round(state[3], 1)])
    #discrete_state =np.array([np.round(state[0]*2)/2, np.round(state[1] *2) /2, np.round(state[2]* 2)/2 ,np.round(state[3]* 2)/2])
    discrete_state = state/np_array_win_size+ np.array([15,10,1,10])
    #discrete_state = (discrete_state/np_array_win_size)
    #print(discrete_state)
    return tuple(discrete_state.astype(int))


print(get_discrete_state([4, -2, 0.418, 0.63751878]))

(55, -10, 42, 16)


## Running

In [118]:
for episode in range(EPISODES): 

    discrete_state = get_discrete_state(env.reset()[0])  
  
    done = False
    episode_reward = 0 
    
    #update every 2000 episodes
    if episode % 2000 == 0: 
        print("Episode: " + str(episode))
    

    while not done: 
        #Always get max for now
        if np.random.random() < epsilon:
            action = np.argmax(q_table[discrete_state]) 
        else:
            action = np.random.randint(0, env.action_space.n) 
        new_env = env.step(action)
        new_state = new_env[0]
        reward = new_env[1]
        done = new_env[2]
        episode_reward += reward 

        new_discrete_state = get_discrete_state(new_state)

        if not done: 
            
            max_future_q = np.max(q_table[new_discrete_state])

            current_q = q_table[discrete_state + (action,)]

            new_q = (1-LEARNING_RATE)*current_q + LEARNING_RATE * (reward + DISCOUNT * max_future_q)
            #new_q = current_q + LEARNING_RATE * (reward + DISCOUNT*(max_future_q) - current_q)
            q_table[discrete_state + (action,)] = new_q

        discrete_state = new_discrete_state

    total_reward += episode_reward

    
    if episode % 1000 == 0: 
        
        mean_reward = total_reward / 1000    
        print("Mean Reward: " + str(mean_reward))
        if (mean_reward < (prev_mean * 0.95) and epsilon > epsilon_min):
            print("Decaying")
            epsilon *= epsilon_decay
        if (mean_reward > (prev_mean * 1.1) and epsilon < epsilon_max):
            print("improving")
            epsilon /= epsilon_decay
        prev_mean = mean_reward
        total_reward = 0


env.close()

Episode: 0
Mean Reward: 0.01
improving
Mean Reward: 10.409
improving
Episode: 2000
Mean Reward: 10.516
Mean Reward: 10.535
Episode: 4000
Mean Reward: 10.538
Mean Reward: 10.579
Episode: 6000
Mean Reward: 10.7
Mean Reward: 10.818
Episode: 8000
Mean Reward: 11.171
Mean Reward: 11.476
Episode: 10000
Mean Reward: 12.119
Mean Reward: 13.016
Episode: 12000
Mean Reward: 13.552
Mean Reward: 14.086
Episode: 14000
Mean Reward: 15.241
Mean Reward: 16.108
Episode: 16000
Mean Reward: 17.03
Mean Reward: 24.618
improving
Episode: 18000
Mean Reward: 27.491
improving
Mean Reward: 31.006
improving
Episode: 20000
Mean Reward: 46.909
improving
Mean Reward: 52.877
improving
Episode: 22000
Mean Reward: 67.434
improving
Mean Reward: 63.083
Decaying
Episode: 24000
Mean Reward: 60.35
Mean Reward: 68.537
improving
Episode: 26000
Mean Reward: 70.759
Mean Reward: 119.898
improving
Episode: 28000
Mean Reward: 146.043
improving
Mean Reward: 197.681
improving
Episode: 30000
Mean Reward: 195.275
Mean Reward: 209.365
